In [17]:
from itertools import product
import os
import pandas as pd

num_reinits = 5
num_total_trials = 25

models = ["kmeans-minibatch", "dbscan", "hac", "hdbscan", "birch"]
output_dir = "/mnt/nfs/scratch1/siddharthami/iesl/clustering_hyperparameters/output/generic"

files = os.listdir(output_dir)
datasets = list(set([ os.path.splitext(output_dir + "/" + x)[0].split("_")[3] for x in files ]))
csv_files = []
final_datasets = []
for dataset in datasets:
    all_yep = { model: False for model in models}
    fnames = []
    for model, rnd in list(product(models, range(num_reinits))):
        fname = 'experiment_' + model + '_' + dataset + '_' + str(rnd) + '.csv'
        if fname in files:
            fnames.append(fname)
            all_yep[model] = True
    if all([all_yep[model] for model in models]):
        csv_files.extend(fnames)
        final_datasets.append(dataset)
print('Datasets', final_datasets, "len", len(final_datasets))
print('CSV files', len(csv_files), csv_files[:30])
df_list = []    
for file in csv_files:
    model = file.split("_")[1]
    dataset = file.split("_")[2]
    reinit_index = file.split("_")[3].split(".")[0]
    
    df = pd.read_csv(output_dir + "/" + file)
    df = df[df['generator_model']=='Sobol']
    df['model'] = [model] * num_total_trials
    df['dataset'] = [dataset] * num_total_trials
    df['reinit_index'] = [reinit_index] * num_total_trials
    df = df.filter(items=['model', 'dataset', 'adjusted_rand_score', 'reinit_index', 'trial_index', 'generator_model'])
    df_list.append(df)

all_df = pd.concat(df_list)
avg_perf_by_model = all_df.groupby(['model', 'dataset']).agg(max_within_dataset=('adjusted_rand_score', 'max')).groupby('model').agg(avg_perf=('max_within_dataset', 'mean'))
all_df.to_pickle("generic_random_df.pkl")
print("Rank average best-performance across datasets by model")
avg_perf_by_model.sort_values(['avg_perf'], ascending=[0])

Datasets ['mfeat-karhunen', 'Bioresponse', 'cnae-9', 'madelon', 'segment', 'ozone-level-8hr', 'car', 'mfeat-fourier', 'pc3', 'wall-robot-navigation', 'splice', 'steel-plates-fault', 'tic-tac-toe', 'dna', 'balance-scale', 'phoneme', 'pc4', 'pc1', 'banknote-authentication', 'ilpd', 'blood-transfusion-service-center', 'kc1', 'mfeat-factors', 'credit-g', 'mfeat-pixel', 'climate-model-simulation-crashes', 'first-order-theorem-proving', 'isolet', 'wdbc', 'kc2', 'kr-vs-kp', 'cmc', 'vowel', 'satimage', 'qsar-biodeg', 'mfeat-zernike', 'wilt', 'diabetes', 'har', 'GesturePhaseSegmentationProcessed', 'vehicle', 'semeion', 'mfeat-morphological'] len 43
CSV files 1016 ['experiment_kmeans-minibatch_mfeat-karhunen_0.csv', 'experiment_kmeans-minibatch_mfeat-karhunen_1.csv', 'experiment_kmeans-minibatch_mfeat-karhunen_2.csv', 'experiment_kmeans-minibatch_mfeat-karhunen_3.csv', 'experiment_kmeans-minibatch_mfeat-karhunen_4.csv', 'experiment_dbscan_mfeat-karhunen_0.csv', 'experiment_dbscan_mfeat-karhunen_

,avg_perf
model,
kmeans-minibatch,0.235206
hac,0.190968
hdbscan,0.174260
dbscan,0.169304
birch,0.169041
